# This is a mostly non-technical demo notebook for a CNN model that predicts March Madness games
##### Below are imports, more than are in use, just extra tools in case.

In [40]:
from __future__ import print_function
import sys
import csv
import tensorflow as tf
import sklearn
import en_core_web_sm
import keras
from keras import layers
from keras import backend as K
from keras.preprocessing.sequence import pad_sequences 
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
from keras_tqdm import TQDMNotebookCallback
from keras.models import Sequential
from keras.layers import Dropout, Activation
from keras.optimizers import SGD
from IPython import display
from keras.preprocessing.sequence import pad_sequences
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#pd.set_option('display.max_columns', None)
pd.options.display.max_columns = 10
pd.set_option('display.max_rows', None)
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'monospace'
from matplotlib.ticker import MaxNLocator
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import os, eli5, shap
np.set_printoptions(threshold=sys.maxsize)
from eli5.sklearn import PermutationImportance
from pdpbox import pdp

In [59]:
#Create dataframes that will be used for performance comparison later
pastModels=[[2017,1,.43857],[2017,2,.44981],[2017,3,.45373],[2017,4,.46100],[2017,5,.46107],[2018,1,.53194],[2018,2,.53693],[2018,3,.54013],[2018,4,.54967],[2018,5,.54987],[2019,1,.41477],[2019,2,.42012],[2019,3,.42698],[2019,4,.42788],[2019,5,.43148]]
cols=['Year','Rank','LogLoss']
pastScores=pd.DataFrame(pastModels, columns=cols)
pastMadness=[[2017,65.7,82,78,68],[2018,57,81,108,95],[2019,74,124,131,99]]
cols=['Year','avg_score','higherSeed_score','myScore','percentile_myScore']
pastMadness=pd.DataFrame(pastMadness, columns=cols)
inputs=39

## Importing the data here and slicing it in different ways for training and testing model
##### We will only be showing testing model performance

In [61]:
tourney_data=pd.read_csv("../Data/TourneyData/tourney_game_master_scaled_all.csv")
removeList=['Ats_pct','Aie','Afour','Aelo','Bts_pct','Bie','Bfour','Belo','ts_pct_diff','ie_diff','four_diff','elo_diff']
if any(elem in tourney_data.columns  for elem in removeList):
    tourney_data=tourney_data.drop(removeList,axis=1)
tourney_data.dropna(subset=['AdjEM_diff'], how='all',inplace=True)
train_data=tourney_data[(tourney_data['Season']<2017)&(tourney_data['Season']>2006)].reset_index(drop=True)
test_data=tourney_data[tourney_data['Season']>2016].reset_index(drop=True)

train_output_data=pd.DataFrame()
test_output_data=pd.DataFrame()
col_names = ['Season','tourn_round','point_diff','ATeamID','ATeam','ASeed','Aregion','BTeamID','BTeam','BSeed','Bregion']
train_other_data=train_data.iloc[:,inputs:inputs+11]
train_output_data['AWon']=train_data.AWon
train_output_data['Season']=train_data.Season
test_other_data=test_data.iloc[:,inputs:inputs+11]
test_output_data['AWon']=test_data.AWon
test_output_data['Season']=test_data.Season
train_data=train_data.iloc[:,:inputs]
test_data=test_data.iloc[:,:inputs]


test_full_data=test_input_data
test_full_data[col_names] = test_other_data 
test_full_data['AWon']=test_output_data.AWon
test_output_data['Season']=test_other_data['Season']

## Here we have a sample of some rows that will be used as inputs to predict the outcome of some March Madness games
##### Does not show all inputs, these inputs are an array of end of season stats for each team and then a comparison differential of said stats
###### Below we also have they testing output data which has the actual outcomes we will be trying to predict

In [45]:
#Show sample of DataFrame
test_input_data.head(5)

,Asos,Amov,APOM,ASAG,AMOR,...,BTeamID,BTeam,BSeed,Bregion,AWon
0,-1.205992,-1.099475,-2.671724,-2.741144,-2.244242,...,arizona,2.0,X,-18,0
1,0.498742,0.379470,0.411734,0.441606,0.399211,...,saint-marys,7.0,X,9,1
2,-1.524222,-1.391061,-0.253325,-0.182857,0.144656,...,arkansas,8.0,Z,-6,0
3,0.056525,0.044417,-0.958692,-1.149769,-2.655446,...,baylor,3.0,W,-18,0
4,-0.777871,-0.841366,-0.414552,-0.303721,-0.207804,...,baylor,3.0,W,-4,0


In [46]:
test_output_data.head(5)



,AWon,Season
0,0,2017
1,1,2017
2,0,2017
3,0,2017
4,0,2017


## Loading Model Framework and Weights
##### This loads a file that has the saved framework and tuned model, it then loads the weights which will be used for predicting games

In [47]:
#Load Model and weights
json_file = open('../winModeling/tournament_win_model_prod.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
optimizer = keras.optimizers.Adagrad(lr=.01,epsilon=0,decay=0)
model.compile(optimizer= optimizer ,loss='binary_crossentropy',metrics=['acc'])
model.load_weights('../winModeling/tournament_win_weights_prod.h5')

print("Loaded model weights from disk")

Loaded model weights from disk


## How we show performance?
##### The primary basis for measuring a model performance is measuring 'Log Loss' or basically a measure of accuracy that factors in confidence
##### Below we will show: Log Loss for random choice of winner/confidence, The Log Loss for the Top 5 models as part of the Kaggle NCAA competition for the 3 years we are testing
##### Link to Kaggle Leaderboard for 2019: https://www.kaggle.com/c/mens-machine-learning-competition-2019/leaderboard

In [48]:
def LogLoss(predictions, realizations):
    predictions_use = predictions.clip(0)
    realizations_use = realizations.clip(0)
    LogLoss = -np.mean( (realizations_use * np.log(predictions_use)) + 
                        (1 - realizations_use) * np.log(1 - predictions_use) )
    return LogLoss

In [49]:
#For reference this is the LogLoss of a model that predicts 50/50 shot of either team winning each time
#rather than favoring one team or another

bench_5050 = np.repeat(0.5, len(test_output_data))
LogLoss(bench_5050, test_output_data.AWon)

0.6931471805599452

In [50]:
#Log Loss for past Kaggle competition winners
pastScores

,Year,Rank,LogLoss
0,2017,1,0.43857
1,2017,2,0.44981
2,2017,3,0.45373
3,2017,4,0.46100
4,2017,5,0.46107
5,2018,1,0.53194
6,2018,2,0.53693
7,2018,3,0.54013
8,2018,4,0.54967
9,2018,5,0.54987


## How does my model stack up?
##### Here is a readout of accuracy followed by Log Loss for the 3 years of Kaggle competition shown above
##### We then show a sample of outputs for the models outputted prediction/confidence

In [62]:
#LogLoss FN for accuracy weighted by confidence


#Accuracy on Test set
years=[2017,2018,2019]
for year in years:
    test_input_temp=test_input_data[test_input_data['Season']==year].reset_index(drop=True)
    test_output_temp=test_output_data[test_output_data['Season']==year].reset_index(drop=True)
    test_output_temp=test_output_temp[['AWon']]
    
    scores = model.evaluate(test_input_temp.iloc[:,0:inputs], test_output_temp, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
    #Test LogLoss on holdout test set
    y_pred_keras = model.predict(test_input_temp.iloc[:,0:inputs]).ravel()
    y_pred_round=np.round(y_pred_keras,0)
    temp = np.array(y_pred_keras)[np.newaxis]
    np.set_printoptions(suppress=True)
    y_pred_tran=temp.T
    print("LL_Test ",year,": ",LogLoss(y_pred_tran, test_output_temp))


y_pred_keras = model.predict(test_input_data.iloc[:,0:inputs]).ravel()
y_pred_round=np.round(y_pred_keras,0)
testPerf=test_input_data.iloc[:,0:inputs]
testPerf['y_pred_confidence']=y_pred_keras

testPerf['y_pred_test']=y_pred_round
testPerf['y_act']=test_output_data.AWon
testPerf.sample(10)

acc: 67.16%
LL_Test  2017 :  AWon    0.584546
dtype: float64
acc: 70.15%
LL_Test  2018 :  AWon    0.584034
dtype: float64
acc: 67.16%
LL_Test  2019 :  AWon    0.524332
dtype: float64


,Asos,Amov,APOM,ASAG,AMOR,Awins_top25,Awins_top5,Awin_pct,AAdjDE,AAdjOE,AAdjEM,AAdjsos,ALuck,Bsos,Bmov,BPOM,BSAG,BMOR,Bwins_top25,Bwins_top5,Bwin_pct,BAdjDE,BAdjOE,BAdjEM,BAdjsos,BLuck,sos_diff,mov_diff,POM_diff,SAG_diff,MOR_diff,wins_top25_diff,wins_top5_diff,win_pct_diff,AdjDE_diff,AdjOE_diff,AdjEM_diff,Adjsos_diff,Luck_diff,y_pred_confidence,y_pred_test,y_act
124,-1.664805,-1.761074,-2.893411,-3.003016,-2.224661,-1.098913,-0.594366,-0.752348,-2.117587,-1.685875,-2.360871,-1.326659,3.098915,1.462180,1.129679,0.819705,0.785599,0.818690,1.051227,1.242486,1.876212,2.038038,0.622339,1.622996,-0.296524,-0.608768,-2.209353,-2.030496,-2.667672,-2.741074,-2.165312,-1.394692,-1.237425,-1.865825,-2.849282,-1.593388,-2.686226,-0.720204,-1.874851,0.087474,0.0,1
101,0.638782,0.768515,0.472194,0.401318,0.438373,0.158626,1.055815,0.340995,-0.099016,0.477517,0.262148,0.537661,-0.249725,0.284002,0.233007,0.645141,0.611270,0.667672,0.628164,1.242486,0.436840,0.438999,0.513686,0.600095,-0.394365,0.353885,0.240998,0.365533,-0.141132,-0.168181,-0.179063,-0.302183,-0.064189,-0.067778,-0.374389,-0.027938,-0.235195,0.657889,-0.088636,0.676542,1.0,0
184,1.211482,1.066883,0.653574,0.643046,0.692928,0.158626,-0.594366,0.584699,1.746456,-0.039163,0.989735,-0.384109,-0.301651,0.785278,1.055584,0.354200,0.320722,0.384515,-1.064086,-0.546476,1.575975,0.062119,0.359011,0.273838,0.719422,-0.700520,0.279968,-0.017501,0.203396,0.222089,0.207153,0.790326,-0.064189,-0.703032,1.126951,-0.276348,0.473754,-0.781340,0.779998,0.714511,1.0,1
134,-1.042908,-1.017338,-0.434705,-0.263433,-0.305710,-0.260554,-0.594366,-0.652568,-0.789388,-0.552179,-0.828998,-0.809681,0.012789,-1.344465,-1.409769,-0.596208,-0.357224,-0.521589,-1.064086,-0.546476,-1.126158,-1.160098,-0.544817,-1.049577,-0.906992,-0.039497,0.238932,0.305798,0.131620,0.077544,0.165772,0.517198,-0.064189,0.335620,0.284912,-0.001859,0.163935,0.076890,0.021348,0.632692,1.0,1
78,-0.594948,-0.433107,0.109434,0.119303,0.242562,0.158626,-0.594366,-1.071642,-1.053316,0.483624,-0.288870,0.013816,-1.003087,1.711595,1.847849,0.683933,0.630640,0.629918,-0.641023,-0.546476,1.311060,-0.017563,1.192595,0.776362,1.374932,-0.286472,-1.643282,-1.625121,-0.428239,-0.384998,-0.289410,0.517198,-0.064189,-1.691579,-0.693647,-0.495314,-0.723685,-0.968338,0.987119,0.254391,0.0,1
52,-2.060314,-1.994766,-3.578624,-3.224600,-3.379948,-1.098913,-0.594366,-1.271201,-1.576473,-2.652265,-2.693156,-1.897097,3.410371,0.112233,0.080894,-2.109102,-2.642869,-3.126639,-1.064086,-0.546476,-0.165400,-1.311863,-1.753497,-1.937224,-0.232148,-2.657369,-1.515576,-1.440441,-0.988097,-0.341635,-0.096302,-0.029056,-0.064189,-0.785592,-0.136134,-0.608667,-0.474139,-1.166365,-0.500353,0.862022,1.0,1
10,0.744107,0.412869,0.673727,0.622902,0.614604,0.577805,1.055815,-0.652568,1.553631,0.110903,0.978132,-0.948180,-1.320760,0.574329,0.588910,0.664537,0.630640,0.573287,-0.641023,1.242486,-0.000269,1.062118,0.505208,0.965138,-0.183720,0.803607,0.105903,-0.136135,-0.011934,-0.023637,0.014045,0.790326,-0.064189,-0.463343,0.296059,-0.274545,-0.006612,-0.534805,0.371136,0.596478,1.0,0
105,0.189723,0.237422,0.331121,0.018583,-0.070736,-1.098913,-0.594366,0.402748,-1.399115,1.195391,-0.013402,1.349864,-0.022535,1.793258,1.834774,0.761517,0.688749,0.686550,0.205102,-0.546476,1.311060,1.696000,0.038761,1.034478,-0.112957,-0.227225,-1.155398,-1.151047,-0.327751,-0.500634,-0.551484,-0.848438,-0.064189,-0.644285,-2.127744,0.796369,-0.716186,1.027768,0.195807,0.248110,0.0,1
104,0.189723,0.237422,0.331121,0.018583,-0.070736,-1.098913,-0.594366,0.402748,-1.399115,1.195391,-0.013402,1.349864,-0.022535,-1.404673,-1.234635,0.082654,0.262612,0.252375,0.628164,-0.546476,-1.726632,0.207898,-0.615753,-0.282572,-1.944885,1.502839,1.140679,1.049812,0.174686,-0.182636,-0.234236,-1.121565,-0.064189,1.511350,-1.083609,1.251005,0.184234,2.330879,-1.164736,0.563984,1.0,1
8,-1.409833,-1.277760,-1.805131,-2.438984,-2.283404,-1.098913,-0.594366,-1.635103,-2.503236,-0.551132,-1.825205,0.32417

# So that's cool, but how do we do at ACTUALLY picking a bracket?
#### We start with the 2017 bracket and show each rounds matchups as my model picked them, followed by the calculated ESPN score

In [55]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from runTournamentNew import runTournament

df=pd.read_csv("../Data/TourneyData/tourney_game_master_scaled_all.csv")
#2017 results
dfList, scoreDF=runTournament(df,2017,'../winModeling/tournament_win_model_prod','../winModeling/tournament_win_weights_prod',True)
for df in dfList:
    print('\n',df)

scoreDF


     ATeamID               ATeam  BTeamID              BTeam
0      1291      mount-st-marys     1437          villanova
1      1458           wisconsin     1439      virginia-tech
2      1438            virginia     1423     unc-wilmington
3      1190   east-tennessee-st     1196            florida
4      1425                 usc     1374                smu
5      1308       new-mexico-st     1124             baylor
6      1266           marquette     1376     south-carolina
7      1407                troy     1181               duke
8      1355     south-dakota-st     1211            gonzaga
9      1321        northwestern     1435         vanderbilt
10     1343           princeton     1323         notre-dame
11     1137            bucknell     1452      west-virginia
12     1268            maryland     1462             xavier
13     1195  florida-gulf-coast     1199         florida-st
14     1388         saint-marys     1433                vcu
15     1315        north-dakota     11

,round,score
0,32,26
1,16,8
2,8,12
3,4,16
4,2,16
5,1,0
6,Total,78


# 2017 could have been better, how about 2018?
#### Same process as before

In [56]:
#2018 results
df=pd.read_csv("../Data/TourneyData/tourney_game_master_scaled_all.csv")
#2017 results
dfList, scoreDF=runTournament(df,2018,'../winModeling/tournament_win_model_prod','../winModeling/tournament_win_weights_prod',True)
for df in dfList:
    print('\n',df)

scoreDF


     ATeamID              ATeam  BTeamID                  BTeam
0      1437          villanova     1347                radford
1      1439      virginia-tech     1104                alabama
2      1293          murray-st     1452          west-virginia
3      1267           marshall     1455             wichita-st
4      1382     st-bonaventure     1196                florida
5      1403         texas-tech     1372       stephen-f-austin
6      1116           arkansas     1139                 butler
7      1168   cal-st-fullerton     1345                 purdue
8      1335               penn     1242                 kansas
9      1301  north-carolina-st     1371             seton-hall
10     1308      new-mexico-st     1155                clemson
11     1120             auburn     1158  college-of-charleston
12     1395                tcu     1393               syracuse
13     1137           bucknell     1277            michigan-st
14     1328           oklahoma     1348           rho

,round,score
0,32,22
1,16,14
2,8,8
3,4,16
4,2,16
5,1,32
6,Total,108


# We did ok considering a 16 seed upset a 1 seed in the 1st round 2018. Finally, this last year:

In [57]:
#2019 results
df=pd.read_csv("../Data/TourneyData/tourney_game_master_scaled_all.csv")
#2017 results
dfList, scoreDF=runTournament(df,2019,'../winModeling/tournament_win_model_prod','../winModeling/tournament_win_weights_prod',True)
for df in dfList:
    print('\n',df)

scoreDF



     ATeamID              ATeam  BTeamID                BTeam
0      1181               duke     1295      north-dakota-st
1      1433                vcu     1416                  ucf
2      1280     mississippi-st     1251              liberty
3      1387        saint-louis     1439        virginia-tech
4      1125            belmont     1268             maryland
5      1261                lsu     1463                 yale
6      1257         louisville     1278            minnesota
7      1133            bradley     1277          michigan-st
8      1211            gonzaga     1192  fairleigh-dickinson
9      1124             baylor     1393             syracuse
10     1293          murray-st     1266            marquette
11     1199         florida-st     1436              vermont
12     1138            buffalo     1113           arizona-st
13     1297  northern-kentucky     1403           texas-tech
14     1305             nevada     1196              florida
15     1285           

,round,score
0,32,23
1,16,24
2,8,20
3,4,16
4,2,16
5,1,32
6,Total,131


# 2019 went pretty well, but how well?
#### Below is my models score compared to the average ESPN score and the score given that you just pick higher seeds
#### Also shown is a percentile score compared to all other users on ESPN
#### Here's a link to the 2019 leaderboard: http://fantasy.espn.com/tournament-challenge-bracket/2019/en/group?groupID=1041234
#### Another note Nate Silver's Fivethirtyeight would only get 72 in 2017, 114 in 2018, and 95 in 2019: https://projects.fivethirtyeight.com/2018-march-madness-predictions/
#### So not only is the model superior on statistical metrics, but it is good at the element humans care about - winning your office pool

In [60]:
pastMadness

,Year,avg_score,higherSeed_score,myScore,percentile_myScore
0,2017,65.7,82,78,68
1,2018,57.0,81,108,95
2,2019,74.0,124,131,99


# Finally, are bigger companies with lots of staff doing better?
#### Not quite, as far as I could find the only company publishing any kind of results is Adobe
#### Their model achieves performance of a reported 98%, but this was just one year. Looking at on year we achieve 99%
#### See this page: https://www.adobe.com/analytics/hack-the-bracket.html?red=a

# Ok, but how does this make anyone money, how about betting the spread?
#### Well, beyond just raising your likelihood to win your office pool, I've also modified the model to predict against the spread
#### So lets load some spread data and the model that predicts against it.

In [74]:

tourney_data=pd.read_csv("../Data/TourneyData/tourney_game_master_scaled_all.csv")
removeList=['Ats_pct','Aie','Afour','Aelo','Bts_pct','Bie','Bfour','Belo','ts_pct_diff','ie_diff','four_diff','elo_diff']
if any(elem in tourney_data.columns  for elem in removeList):
    tourney_data=tourney_data.drop(removeList,axis=1)
tourney_data.dropna(subset=['AdjEM_diff'], how='all',inplace=True)
tourney_data.dropna(subset=['line'], how='all',inplace=True)
train_data=tourney_data[(tourney_data['Season']<2016)&(tourney_data['Season']>2003)].reset_index(drop=True)
test_data=tourney_data[tourney_data['Season']>2015].reset_index(drop=True)

#train_output_data=pd.DataFrame()
#test_output_data=pd.DataFrame()
col_names = ['Season','tourn_round','point_diff','ATeamID','ATeam','ASeed','Aregion','BTeamID','BTeam','BSeed','Bregion']
train_other_data=train_data.iloc[:,inputs:inputs+11]
train_output_data['ASpread']=train_data.ASpread
train_output_data['Season']=train_data.Season
test_other_data=test_data.iloc[:,inputs:inputs+11]
test_output_data['ASpread']=test_data.ASpread
train_input_data=train_data.iloc[:,:inputs]
test_input_data=test_data.iloc[:,:inputs]
test_full_data=test_input_data.copy()
test_full_data[col_names] = test_other_data.copy()
test_full_data['ASpread']=test_output_data['ASpread'].copy()




json_file = open("../spreadModeling/tournament_spread_model_prod.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
optimizer = keras.optimizers.Adagrad(lr=.001,epsilon=0,decay=0)
model.compile(optimizer= optimizer ,loss='binary_crossentropy',metrics=['acc'])
model.load_weights('../spreadModeling/tournament_spread_weights_prod.h5')

print("Loaded model weights from disk")



Loaded model weights from disk


# How do we compare performance for spread?
#### Well because we are trying to beat vegas in this instance an average performance would be 50/50 because of how they try to set the lines
#### With that being said a good log loss would be anything less that .69 and a good accuracy would obviously be better than 50%

In [76]:


years=[2016,2017,2018]
for year in years:
    test_input_temp=test_full_data[test_full_data['Season']==year].reset_index(drop=True)
    test_output_temp=test_full_data[test_full_data['Season']==year].reset_index(drop=True)
    test_output_temp=test_output_temp[['ASpread']]
    
    scores = model.evaluate(test_input_temp.iloc[:,0:inputs], test_output_temp, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

    #Test LogLoss on holdout test set
    y_pred_keras = model.predict(test_input_temp.iloc[:,0:inputs]).ravel()
    y_pred_round=np.round(y_pred_keras,0)
    temp = np.array(y_pred_keras)[np.newaxis]
    np.set_printoptions(suppress=True)
    y_pred_tran=temp.T
    print("LL_Test ",year,": ",LogLoss(y_pred_tran, test_output_temp))




#show pred df for last season
y_pred_keras = model.predict(test_input_data.iloc[:,0:inputs]).ravel()
y_pred_round=np.round(y_pred_keras,0)
testPerf=test_input_data.iloc[:,0:inputs]
testPerf['y_pred_confidence']=y_pred_keras

testPerf['y_pred_test']=y_pred_round
testPerf['y_act']=test_output_data.ASpread

testPerf.sample(10)

acc: 67.69%
LL_Test  2016 :  ASpread    0.65695
dtype: float64
acc: 52.24%
LL_Test  2017 :  ASpread    0.701815
dtype: float64
acc: 64.18%
LL_Test  2018 :  ASpread    0.668318
dtype: float64


,Asos,Amov,APOM,ASAG,AMOR,Awins_top25,Awins_top5,Awin_pct,AAdjDE,AAdjOE,AAdjEM,AAdjsos,ALuck,Bsos,Bmov,BPOM,BSAG,BMOR,Bwins_top25,Bwins_top5,Bwin_pct,BAdjDE,BAdjOE,BAdjEM,BAdjsos,BLuck,sos_diff,mov_diff,POM_diff,SAG_diff,MOR_diff,wins_top25_diff,wins_top5_diff,win_pct_diff,AdjDE_diff,AdjOE_diff,AdjEM_diff,Adjsos_diff,Luck_diff,y_pred_confidence,y_pred_test,y_act
182,-0.707631,-0.646100,-1.324682,-1.282199,-1.403553,-0.884110,-0.487945,-0.099096,-0.335889,-1.751944,-1.354867,-1.681564,1.584358,-0.051118,-0.017612,0.777556,0.672479,0.743592,0.978733,1.410179,0.295938,0.721236,0.574341,0.802957,-0.686781,0.643023,-0.453098,-0.427580,-1.789040,-1.667219,-1.801088,-1.092224,-1.144929,-0.323390,-0.754827,-1.634219,-1.496829,-0.692316,-1.994886,0.393901,0.0,0
6,-0.267872,-0.196477,0.523253,0.470271,0.497736,0.978733,1.410179,0.295938,0.517069,-0.001022,0.302955,-0.470193,-0.389709,1.213858,1.146801,0.489346,0.588225,0.612469,-0.884110,-0.487945,0.536812,-0.129326,0.844792,0.482272,0.838593,0.342090,-1.005063,-0.899023,0.017748,-0.112616,-0.105482,1.094675,1.124575,-0.218331,0.476400,-0.595162,-0.120536,-0.906420,0.078987,0.469972,0.0,0
79,1.461734,1.830803,0.879277,0.891537,0.858325,1.910155,-0.487945,0.368484,1.294767,0.850198,1.322046,0.298531,-0.727749,3.327436,3.667013,0.947092,0.942089,0.923887,1.910155,-0.487945,2.454171,1.678007,1.566276,2.020196,2.078980,-0.561591,-1.261891,-1.224139,-0.068290,-0.055566,-0.064459,0.001226,-0.010177,-1.475564,-0.267774,-0.504108,-0.481448,-1.228023,1.210930,0.422274,0.0,0
189,-1.408523,-1.505540,-2.155405,-2.225837,-1.567457,-0.884110,-0.487945,-0.519615,-1.675330,-1.359866,-1.882310,-1.167008,2.340631,1.860492,1.477702,0.947092,0.925239,0.956668,1.444444,1.410179,2.145852,2.301876,0.859658,1.919616,-0.116482,-0.366274,-2.200527,-1.980669,-2.635075,-2.679850,-2.115595,-1.365586,-1.144929,-1.870681,-2.865328,-1.559290,-2.640502,-0.730325,-1.764231,0.387791,0.0,0
77,-0.452925,-0.431148,-0.866937,-0.945186,-0.698765,-0.884110,-0.487945,0.388434,-1.532755,-0.316915,-1.109386,-0.137434,1.443671,0.451765,0.776550,0.641928,0.621927,0.694421,1.444444,-0.487945,0.295938,0.473156,0.895132,0.869259,-0.053958,0.665980,-0.619097,-0.809601,-1.287154,-1.339183,-1.172073,-1.365586,-0.010177,0.008859,-1.440567,-0.852199,-1.372188,-0.070942,-1.887564,0.396445,0.0,1
187,0.685197,0.626303,0.760603,0.739881,0.809154,1.444444,-0.487945,-0.012381,0.835828,0.437679,0.779941,-0.473394,-0.834920,0.181940,0.442430,-0.917797,-1.063140,-1.485505,-0.884110,-0.487945,0.727585,-0.558229,-1.090597,-1.048342,-0.393952,-1.555005,0.322639,0.108216,1.408687,1.513300,1.904632,1.368038,-0.010177,-0.558460,1.016775,1.071077,1.275995,-0.068191,2.214215,0.614584,1.0,0
36,0.283823,0.354428,0.591067,0.689329,0.661640,0.978733,3.308303,-0.320700,-0.617578,1.747872,0.792308,0.925673,-1.039876,0.230359,0.297191,0.811463,0.706180,0.809154,0.978733,-0.487945,0.912576,-0.068704,1.383667,0.873958,1.148825,0.266788,0.021786,0.024692,-0.197346,-0.027041,-0.132830,0.001226,2.259327,-0.894649,-0.387487,0.254082,-0.052596,-0.166180,0.740318,0.457010,0.0,0
38,0.230359,0.297191,0.811463,0.706180,0.809154,0.978733,-0.487945,0.912576,-0.068704,1.383667,0.873958,1.148825,-0.266788,-3.057738,-2.886611,-3.579502,-3.557039,-2.763957,-0.884110,-0.487945,-3.095571,-2.288972,-2.432903,-2.951553,-2.279589,-2.430834,2.185339,2.086832,3.703021,3.595612,2.971223,1.094675,-0.010177,2.677359,1.613923,2.677099,2.665311,2.323640,2.494709,0.622497,1.0,1
133,0.241671,0.512276,0.692789,0.402868,0.612469,0.047312,-0.487945,0.536812,-0.204796,1.141427,0.633986,0.715946,-0.417781,-0.642923,-0.583980,-1.070379,-0.928335,-0.862669,-0.884110,-0.487945,0.465016,-1.647956,-0.262182,-1.140843,-0.237664,-1.550469,0.577705,0.709982,1.480385,1.113952,1.220920,0.547951,-0.010177,-0.005248,1.052265,0.983580,1.238811,0.636195,1.829819,0.612728,1.0,0
35,0.230359,0.297191,0.811463,0.706180,0.809154,0.978733,-0.487945,0.912576,-0.068704,1.383667,0.873958,1.148825,-0.266788,0.366732,0.490

## This average of 61.4% accuracy tells us over the long run we could stand to make a bit of money in Vegas, we could maybe even eke out extra performance if we focused on picks our model was most confident in